# <center>Model Training (v1)</center>

<br>
<br>
<p>As a first step, we need to run the previously developed code.</p>
<br>
<br>

In [ ]:

!wget -O trainingandtestdata.zip http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
print('unziping ...')
!unzip -o -j trainingandtestdata.zip


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


data = pd.read_csv("training.1600000.processed.noemoticon.csv", header=None, encoding='ISO-8859-1')
test = pd.read_csv("testdata.manual.2009.06.14.csv", header=None, encoding='ISO-8859-1')


data.columns = ["target", "ids", "date", "flag", "user", "text"]
test.columns = ["target", "ids", "date", "flag", "user", "text"]


data["target"] = data["target"].replace(4, 1)
test["target"] = test["target"].replace(4, 1)


df = data[["target", "text"]]
ts = test[["target", "text"]]


ts_bin = ts[ts["target"]!=2]
ts_neut = ts[ts["target"]==2]




df.to_csv('training_data.csv')
ts_bin.to_csv('test_data.csv')
ts_neut.to_csv('neutral_data.csv')

In [2]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD




df_m = pd.read_csv("training_data.csv")




labels = df_m["target"]
tweets = df_m["text"]


vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(tweets)
print(X.shape)


(1600000, 684047)


In [3]:

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=2)


In [4]:
scaler = StandardScaler(with_mean=False)

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [5]:
svd = TruncatedSVD(n_components=100)
svd.fit(X_train)


X_train_svd = svd.transform(X_train)
X_test_svd = svd.transform(X_test)

<br>
<br>
<p>Here we will train and test the first model.</p>
<br>
<br>

In [6]:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics


logReg = LogisticRegression()
logReg.fit(X_train_svd, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [7]:
predicted = logReg.predict(X_test_svd)

In [10]:
print("Accuracy for Logistic Regression Classifier: " + str(metrics.accuracy_score(y_test, predicted)*100)+"%")

Accuracy for Logistic Regression Classifier: 51.69625%
